In [6]:
import numpy as np

SAMPLE_RATE = 16000
CHUNK_LENGTH= 30
def chunk_and_resample_audio(
    audio: np.ndarray,
    audio_sample_rate: int,
    model_sample_rate=SAMPLE_RATE,
    model_chunk_seconds=CHUNK_LENGTH,
) -> list[np.ndarray]:
    if audio_sample_rate != model_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=audio_sample_rate, new_freq=model_sample_rate)
        audio = resampler(audio)
        if audio.shape[0] > 1:
            audio = torch.mean(audio, dim=0)
        audio = audio.numpy()
        audio_sample_rate = model_sample_rate

    number_of_full_length_audio_chunks = (
        audio.shape[0] // audio_sample_rate // model_chunk_seconds
    )
    last_sample_in_full_length_audio_chunks = (
        audio_sample_rate * number_of_full_length_audio_chunks * model_chunk_seconds
    )

    if number_of_full_length_audio_chunks == 0:
        return [audio]

    return [
        *np.array_split(
            audio[:last_sample_in_full_length_audio_chunks],
            number_of_full_length_audio_chunks,
        ),
        audio[last_sample_in_full_length_audio_chunks:],
    ]


In [19]:
import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torchaudio
processor = AutoProcessor.from_pretrained("openai/whisper-base.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base.en").to("cuda")
print(model.device)
def get_transcript(audio_path):

  waveform, sample_rate = torchaudio.load(audio_path)
  audios=chunk_and_resample_audio(waveform,sample_rate)

  transcripts=[]
  for i,audio in enumerate(audios):
    print("start time:",i*30,"s")
    end_time= min((i+1)*30,i*30+ audio.shape[0]/SAMPLE_RATE)
    print("end time:",end_time)
    print(audio.shape)
    input_features = processor(audio, return_tensors="pt").input_features.to("cuda")
    print("fetaure shape:",input_features.shape)
    generated_ids = model.generate(inputs=input_features)
    print("gen ids",generated_ids.shape)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    cur_obj={
        "start_time":i*30,
        "end_time":end_time,
        "transcription":transcription
    }
    transcripts.append(cur_obj)
  return transcripts
trans=get_transcript("/content/audio/AlphaGo & AlphaGo Zero (DL 24).wav")

cuda:0


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

start time: 0 s
end time: 30
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:655: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 80])
start time: 30 s
end time: 60
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


...

gen ids torch.Size([1, 84])
start time: 1290 s
end time: 1320
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 80])
start time: 1320 s
end time: 1350
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 74])
start time: 1350 s
end time: 1361.21175
(179388,)
fetaure shape: torch.Size([1, 80, 3000])
gen ids torch.Size([1, 28])


In [23]:
import json
import os
audio_folder= "/content/audio"
audio_dict={}
for audio_path in os.listdir(audio_folder):
  if not audio_path.endswith(".wav") or (os.path.splitext(audio_path)[0] in audio_dict.keys()):
    continue
  print(audio_path)
  pred=get_transcript(os.path.join(audio_folder,audio_path))
  audio_dict[os.path.splitext(audio_path)[0]]=pred
  print(os.path.splitext(audio_path)[0].replace(" ",""))
with open("trans_playlist.json","w") as f:
  json.dump(audio_dict,f)



Out-of-Sample Validation (DL 06).wav


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

start time: 0 s
end time: 30
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:655: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 86])
start time: 30 s
end time: 60
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


...

gen ids torch.Size([1, 76])
start time: 900 s
end time: 930
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 68])
start time: 930 s
end time: 960
(480000,)
fetaure shape: torch.Size([1, 80, 3000])


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


gen ids torch.Size([1, 52])
start time: 960 s
end time: 979.1274375
(306039,)
fetaure shape: torch.Size([1, 80, 3000])
gen ids torch.Size([1, 54])
Feed-ForwardNeuralNetworks(DL07)


In [24]:
len(audio_dict.keys())

26